In [1]:
import pandas as pd
import random
import numpy as np
import seaborn as sns
import math
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression,Lasso
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

In [2]:
X=pd.read_csv('x_train.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
X_test=pd.read_csv('x_test.csv',delimiter=";",header=None,names=['f'+str(i) for i in range (223)])
y=pd.read_csv('y_train.csv',names=['class'])
y=y['class']

In [3]:
X_ftr=pd.read_csv('ftr_train.csv')
X_ftr.drop('Unnamed: 0',inplace=True,axis=1)

In [8]:
# Фуникция считает качество модели на кроссвалилации
# clf - модель, X,y - данные, ftr_set - пподмножество названий колонок, на которых считаем модель
# cv_val - количество холдов для кросс валидации
def CV_score(clf,X,y,ftr_set,cv_val):
    # Если подмоножество признаков пустое, то возвращаем 0 
    if len(ftr_set)==0:
        return 0
    # Иначе делаем кроссвалидацию
    scores = cross_val_score(clf, X[ftr_set],y,cv=cv_val)
    return scores.mean()
    #return random.random()

# Функция определяет  фичу, добавление которой наилучшим образом увеличивает качество 
# Фуникция берет случайную фичу из диапазона, добавляет её к стартовому набору фич и строит качество
# N -  максимальное количество итераций
# start_ftr - стартовый набор признаков
# start_cv - стартовое значение качества
def get_best_ftr(clf,X,y,start_ftr,start_cv,cv_val,N_params):
    print ("===START ftr lookup")
    print ("Start_ftr_set=",start_ftr)
    print ("Start_cv=     ",start_cv)
    i=0
    max_iter=N_params[0]
    start_ftr_N=N_params[1]
    end_ftr_N=N_params[2]
    used_ftr=start_ftr.copy()
    best_cv=start_cv
    best_ftr=''
    feature_names=list(X.columns)
    #while i<max_iter:
    for ftr_index in range(max_iter):
        # выбираем случайную фичу
        #ftr_index=random.randint(start_ftr_N,end_ftr_N)
        ftr_name=feature_names[ftr_index]
        # Проверяем, что её не использвоали
        if ftr_name in used_ftr:
            continue
        # Считаем качество на наборе фич с учетом выбранной
        ftr_set=start_ftr.copy()
        ftr_set.append(ftr_name)
        cv=CV_score(clf,X,y,ftr_set,cv_val)
        if cv>best_cv:
            best_cv=cv
            best_ftr=ftr_name
            print ("  ! ",end='')
        else:
            print ("    ",end='')
        print ("{:5} {:10}".format(i,ftr_name), " ",cv)
        used_ftr.append(ftr_name)
        i=i+1
    print ("===END ftr lookup ")
    return best_ftr,best_cv

# Фуникция производит отбор фич методом Sequential Feature Selection
def sfs(clf,X,y,start_ftr,cv_val,N_params):
    cur_ftr=start_ftr.copy()
    best_cv=CV_score(clf,X,y,start_ftr,cv_val)
    doAdd=True
    while doAdd:
        new_ftr,best_cv=get_best_ftr(clf,X,y,cur_ftr,best_cv,cv_val,N_params)
        if (new_ftr!=''):
            print (new_ftr+" added! New cv=",best_cv)
            cur_ftr.append(new_ftr)
        else: doAdd=False
    print ("Final_ftr_set=",cur_ftr, " final_cv=",best_cv)
    return cur_ftr,best_cv

In [10]:
clf = RandomForestClassifier(max_features='auto',n_estimators=300,max_depth=100,random_state=42)
start_ftr=['f11', 'f96']
max_iter=223
start_ftr_N=0
end_ftr_N=223
N_params=(max_iter, start_ftr_N,end_ftr_N)
best_ftr,best_cv=sfs(clf,X,y,start_ftr,5,N_params)

===START ftr lookup
Start_ftr_set= ['f11', 'f96']
Start_cv=      0.472915735397
  !     0 f0           0.542557493827
        1 f1           0.509307649788
        2 f2           0.510168872979
        3 f3           0.460002084655
        4 f4           0.45312973476
        5 f5           0.509314193207
        6 f6           0.540566515606
        7 f7           0.518759148477
        8 f8           0.462023802257
        9 f9           0.452566094683
       10 f10          0.473787644807
       11 f12          0.506741571333
       12 f13          0.526223018967
       13 f14          0.440799327421
       14 f15          0.468013093715
       15 f16          0.458869887843
  !    16 f17          0.557477379262
       17 f18          0.533972945424
       18 f19          0.486386572173
       19 f20          0.469183030329
       20 f21          0.456290690778
       21 f22          0.527093311088
       22 f23          0.533972971372
       23 f24          0.458303381857
       24

In [6]:
Получится ли 156?

'f222f222'